In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
path_to_zip = '/content/drive/MyDrive/Samal_work/archive.zip'
tar = zipfile.ZipFile(path_to_zip)
tar.extractall()
tar.close()

In [3]:
import os 
import glob 

from PIL import Image, ImageDraw
%matplotlib inline 
from matplotlib import pyplot as plt # this lets you draw inline pictures in the notebooks
import pylab # this allows you to control figure size 
pylab.rcParams['figure.figsize'] = (10.0, 8.0) # this controls figure size in the notebook
import cv2
import numpy as np
from collections import Counter 
from pathlib import Path
import pandas as pd
import torch
import tqdm
from matplotlib.patches import Rectangle
import pickle

from math import cos, sin, sqrt

 # SynergyNet importion

In [ ]:
print('\n> Cloning the repo')
!git clone https://github.com/choyingw/SynergyNet

%cd SynergyNet


> Cloning the repo
fatal: destination path 'SynergyNet' already exists and is not an empty directory.
/content/SynergyNet/SynergyNet


In [ ]:
%cd Sim3DR
!sh ./build_sim3dr.sh

%cd ../FaceBoxes
!sh ./build_cpu_nms.sh
%cd ..

# !gdown --id 1SQsMhvAmpD1O8Hm0yEGom0C0rXtA0qs8
# !unzip -n 3dmm_data
!unzip -n /content/drive/MyDrive/Samal_work/3dmm_data.zip

!gdown --id 1BVHbiLTfX6iTeJcNbh-jgHjWDoemfrzG

%ls

/content/SynergyNet/SynergyNet/Sim3DR
running build_ext
skipping 'lib/rasterize.cpp' Cython extension (up-to-date)
/content/SynergyNet/SynergyNet/FaceBoxes
running build_ext
skipping 'nms/cpu_nms.c' Cython extension (up-to-date)
/content/SynergyNet/SynergyNet
Archive:  /content/drive/MyDrive/Samal_work/3dmm_data.zip
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1BVHbiLTfX6iTeJcNbh-jgHjWDoemfrzG
To: /content/SynergyNet/SynergyNet/best.pth.tar
100% 75.4M/75.4M [00:00<00:00, 212MB/s]
3dmm_data/              img/                 singleImage.py
artistic.py             LICENSE              singleImage_simple.py
backbone_nets/          loss_definition.py*  synergy3DMM.py
benchmark_aflw2000.py*  main_train.py        synergy_demo.ipynb
benchmark.py*           model_building.py*

In [ ]:
import torch
import torchvision.transforms as transforms
import numpy as np
import cv2
from utils.ddfa import ToTensor, Normalize
# from model_building import SynergyNet
from utils.inference import crop_img, predict_sparseVert, draw_landmarks, predict_denseVert, predict_pose, draw_axis
import argparse
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import os
import os.path as osp
import glob
from FaceBoxes import FaceBoxes
from utils.render import render
import scipy.io as sio

In [ ]:
from model_building import SynergyNet

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-f', '--files', default='./img/', help='path to a single image or path to a folder containing multiple images')
parser.add_argument("--png", action="store_true", help="if images are with .png extension")
parser.add_argument('--img_size', default=120, type=int)
parser.add_argument('-b', '--batch-size', default=1, type=int)

args = parser.parse_args()
# args.files = data_biwi_dict['rel_path'][:100]
args.files = '/content/faces_0/01/frame_00003_rgb.png'

In [ ]:
# Following 3DDFA-V2, we also use 120x120 resolution
IMG_SIZE = 120

# load pre-tained model
checkpoint_fp = '/content/drive/MyDrive/Samal_work/best_pose.pth.tar' 
args.arch = 'mobilenet_v2'
args.devices_id = [0]

checkpoint = torch.load(checkpoint_fp, map_location=lambda storage, loc: storage)['state_dict']
    
model = SynergyNet(args)
model_dict = model.state_dict()

# because the model is trained by multiple gpus, prefix 'module' should be removed
for k in checkpoint.keys():
    model_dict[k.replace('module.', '')] = checkpoint[k]

model.load_state_dict(model_dict, strict=False)
# model = model.cuda()
model.eval()

# face detector
face_boxes = FaceBoxes()

# preparation
transform = transforms.Compose([ToTensor(), Normalize(mean=127.5, std=128)])

In [ ]:
def _to_ctype(arr):
    if not arr.flags.c_contiguous:
        return arr.copy(order='C')
    return arr
tri = sio.loadmat('./3dmm_data/tri.mat')['tri'] - 1

# Make CSV

In [ ]:
class DatasetCSV():
  def __init__(self, path, args, checkpoint_fp='/content/drive/MyDrive/Samal_work/best_pose.pth.tar'):
    self.dataset = {}
    self.dataset['uid'] = None
    self.dataset['img_name'] = None
    self.dataset['rel_path'] = None
    self.dataset['resolution'] = None
    self.dataset['facerect'] = None
    self.dataset['landmarks'] = None
    self.dataset['pose_angles'] = None
    self.path = path
    self.args_model = args
    self.imgs_path = self.get_files(self.path) # default end_path='.png'

    # self.dataset['img_name'] = self.get_img_name(self.imgs_path)
    self.dataset['rel_path'] = self.get_rel_path(self.imgs_path)
    # self.dataset['uid'] = self.get_uid(self.imgs_path)
    # self.dataset['resolution'] = self.get_resolution()
    self.dataset['facerect'], self.indexes = self.get_facerect() #indexes for facerects 
    # self.dataset['landmarks'] = self.get_landmarks()
    # self.dataset['pose_angles'] = self.get_poses()

  def get_files(self, path='./', end_path='/*.png', ext=('.info')):
    """ Get all image files """
    path_sorted = sorted(os.listdir(path))
    files_all = []
    for file in path_sorted:
       files_path = os.path.join(path, file)
       files_list = glob.glob(f'{files_path}{end_path}')
       files_all.extend(files_list)
    return files_all
  def get_img_name(self, paths):
    res_img_name = []
    for path in paths:
      res_img_name.append(Path(path).stem)
    return res_img_name
  def get_rel_path(self, paths):
    res_rel_path = []
    for path in paths:
      # res_rel_path.append(path[9:]) #delete '/content/'
      res_rel_path.append("/".join(path.strip("/").split('/')[1:])) #delete first directory in path
    return res_rel_path
  def get_uid(self, paths):
    pathC = Counter(map(os.path.dirname, paths))
    res_uid = []
    for i, key in enumerate(pathC.keys()):
      res_uid.extend([i for n in range(pathC[key])])
    return res_uid
  def get_resolution(self):
    res_resolution = []
    for img_path in self.dataset['rel_path']:
      img = cv2.imread(img_path)
      h, w, d = img.shape
      res_resolution.append((h,w))
    return res_resolution
  def make_correct_rect(curr_rect): # x, y, w, h
    xmin, ymin, xmax, ymax, score = curr_rect
    w = xmax - xmin + 1
    h = ymax - ymin + 1
    return (xmin, ymin, w, h)
  def find_center_point(img_path):
    img_cent = cv2.imread(img_path)
    width, depth = np.shape(img_cent)[0], np.shape(img_cent)[1]
    x_center = width / 2
    y_center = depth / 2
    return (x_center, y_center)
  def make_list_of_rects(rects):
    x_cords = []
    y_cords = []
    for rect in rects:
      correct_rect = self.make_correct_rect(rect)
      x_cords.append(correct_rect[0])
      y_cords.append(correct_rect[1])
    A = np.column_stack((x_cords, y_cords))
    return A
  def get_facerect(self):
    files = self.dataset['rel_path']
    res_facerect = []
    indexes = {}
    for img_fp in tqdm.tqdm(files):
      img_ori = cv2.imread('/content/'+ str(img_fp))
      try:
        # crop faces
        rects = face_boxes(img_ori)
        if len(rects) == 1:
          res_facerect.append(self.make_correct_rect(rects[0]))
          indexes[img_fp] = 0
        else:
          center_of_img = self.find_center_point(img_fp)
          A = self.make_list_of_rects(rects)
          distances = np.linalg.norm(A-center_of_img, axis=1)
          min_index = np.argmin(distances)
          res_facerect.append(self.make_correct_rect(rects[min_index]))
          indexes[img_fp] = min_index
      except:
        res_facerect.append([None])
        indexes[img_fp] = None
    return res_facerect, indexes
def make_seven_landmarks(pts):
  y_cord, x_cord = [], []
  # nose, left_eye_left, left_eye_right, 
  # right_eye_left, right_eye_right, 
  # mouth_left, mouth_right
  # list_of_vert = [33, 36, 39, 42, 45, 48, 54]
  list_of_vert = [30, 36, 39, 42, 45, 48, 54]
  for ind in list_of_vert:
    x_cord.append(pts[0][ind])
    y_cord.append(pts[1][ind])
  finall_landms = tuple((x, y) for x, y in zip(x_cord, y_cord))
  return finall_landms
def get_landmarks(self):
  rects = self.dataset['facerect']
  res_landmarks = []
  for ind, img_fp in enumerate(self.indexes.keys()):
    img_ori = cv2.imread('/content/'+ str(img_fp))
    roi_box = rects[ind]
    if roi_box != [None]:
      # enlarge the bbox a little and do a square crop
      HCenter = (rect[1] + rect[3])/2
      WCenter = (rect[0] + rect[2])/2
      side_len = roi_box[3]-roi_box[1]
      margin = side_len * 1.2 // 2
      roi_box[0], roi_box[1], roi_box[2], roi_box[3] = WCenter-margin, HCenter-margin, WCenter+margin, HCenter+margin
      img = crop_img(img_ori, roi_box)
      img = cv2.resize(img, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
      input = transform(img).unsqueeze(0)
      with torch.no_grad():
            # input = input.cuda()
            param = model.forward_test(input)
            param = param.squeeze().cpu().numpy().flatten().astype(np.float32)

      # inferences
      lmks = predict_sparseVert(param, roi_box, transform=True)
      res_landmarks.append(self.make_seven_landmarks(lmks))
    else:
      res_landmarks.append([None])
  return res_landmarks
def make_pose_by_path(pose_path):
  pose_path = '/content/'+pose_path[:-8] + '_pose.txt'
  # print(pose_path)
  # Load pose in degrees
  pose_annot = open(pose_path, 'r')
  R = []
  for line in pose_annot:
   line = line.strip('\n').split(' ')
   L = []
   if line[0] != '':
     for nb in line:
       if nb == '':
         continue
       L.append(float(nb))
     R.append(L)
  R = np.array(R)
  # print(R)
  T = R[3,:]
  R = R[:3,:]
  pose_annot.close()
  R = np.transpose(R)


  roll = -np.arctan2(R[1][0], R[0][0]) * 180 / np.pi
  yaw = -np.arctan2(-R[2][0], np.sqrt(R[2][1] ** 2 + R[2][2] ** 2)) * 180 / np.pi
  pitch = np.arctan2(R[2][1], R[2][2]) * 180 / np.pi
  # print(yaw, pitch, roll) # for plot it is correct
  return (roll, pitch, yaw)
def get_poses(self):
  files = self.dataset['rel_path']
  res_poses = []
  for img_fp in tqdm.tqdm(files):
    pose = make_pose_by_path(img_fp)
    res_poses.append(pose)
  return res_poses

In [ ]:
ABS_PATH = '/content/faces_0'
test_dataset_class = DatasetCSV(ABS_PATH, args)


100%|██████████| 15678/15678 [15:54<00:00, 16.43it/s]


In [ ]:
dataset_dict = test_dataset_class.dataset
print(dataset_dict.keys())

dict_keys(['uid', 'img_name', 'rel_path', 'resolution', 'facerect', 'landmarks', 'pose_angles'])


In [ ]:
print(len(dataset_dict['facerect']))

15678


In [ ]:
print(dataset_dict['facerect'][1:10])

[[None], [None], [None], [None], [None], [None], [None], [None], [None]]
